# VectorStores

Uma das maneiras mais comuns de armazenar e buscar dados não estruturados é realizando o embedding e armazenando os vetores resultantes e, em seguida, na hora da consulta, realizar o embedding da consulta e recuperar os vetores 'mais semelhantes'. Uma VectorStore faz o armazenamento dos vetores e a realização da busca de vetores para você

## Chroma VectorStore

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Document Laoding

In [ ]:
caminho = "arquivos/Explorando o Universo das IAs com Hugging Face.pdf"
loader = PyPDFLoader(caminho)
paginas = loader.load()

In [ ]:
len(paginas)

### Text Splitting

In [ ]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)
len(documents)

### Criando a VectorStore

In [ ]:
#!pip install pysqlite3-binary

In [ ]:
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [ ]:
from langchain_chroma import Chroma

diretorio = 'arquivos/chroma_vectorstore'

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

In [ ]:
print(vectorstore._collection.count())

### Importando vectorstore do disco

In [ ]:
diretorio = 'arquivos/chroma_vectorstore'

vectorstore = Chroma(
    embedding_function=embeddings_model,
    persist_directory=diretorio
)

### Retrieval

In [ ]:
pergunta = 'O que é o Hugging Face?'

docs = vectorstore.similarity_search(pergunta, k=5)
len(docs)

In [ ]:
for doc in docs:
    print(doc.page_content)
    print(f'====== {doc.metadata}\n\n')

## FAISS VectorStore

https://python.langchain.com/docs/integrations/vectorstores/

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
caminho = "arquivos/Explorando o Universo das IAs com Hugging Face.pdf"
loader = PyPDFLoader(caminho)
paginas = loader.load()

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores.faiss import FAISS

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings_model
)

In [ ]:
pergunta = 'O que é o Hugging Face?'

docs = vectorstore.similarity_search(pergunta, k=5)
len(docs)

In [ ]:
for doc in docs:
    print(doc.page_content)
    print(f'====== {doc.metadata}\n\n')

### Salvando bd FAISS

In [ ]:
vectorstore.save_local('arquivos/faiss_bd')

### Importando bd FAISS

In [ ]:
from langchain_community.vectorstores.faiss import FAISS

vectorstore = FAISS.load_local(
    'arquivos/faiss_bd',
    embeddings=embeddings_model,
    allow_dangerous_deserialization=True
)

### Algumas limitações